In [1]:
# task2.py を逐次実行してみて中身を把握する

import sys
import os

import numpy as np
import argparse
import textwrap


import librosa
import librosa.display
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.axes_grid1 import make_axes_locatable

sys.path.append('/store/projects/ml/mathG/DCASE2017/baseline/DCASE2017-baseline-system/')

from dcase_framework.application_core import BinarySoundEventAppCore
from dcase_framework.parameters import ParameterContainer
from dcase_framework.utils import *


In [2]:
project_base=os.path.dirname(os.path.realpath('../../../baseline/DCASE2017-baseline-system/applications/system/'))


params = ParameterContainer(
    project_base=project_base,
    path_structure={
        'feature_extractor': [
            'dataset',
            'feature_extractor.parameters.*'
        ],
        'feature_normalizer': [
            'dataset',
            'feature_extractor.parameters.*'
        ],
        'learner': [
            'dataset',
            'feature_extractor',
            'feature_stacker',
            'feature_normalizer',
            'feature_aggregator',
            'learner'
        ],
        'recognizer': [
            'dataset',
            'feature_extractor',
            'feature_stacker',
            'feature_normalizer',
            'feature_aggregator',
            'learner',
            'recognizer'
        ],
    }
)

params

{}

In [3]:
params.load(filename=project_base + '/parameters/task2.defaults.yaml')
params['active_set'] = 'dcase2017_gpu'
params.process() # 実行用のパラメータ構造に変える？


setup_logging(parameter_container=params['logging'])

#app.show_parameters()

/store/projects/ml/mathG/DCASE2017/baseline/DCASE2017-baseline-system/dcase_framework/files.py:379: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  dict.update(self, yaml.load(infile))


In [4]:
class Task2AppCore(BinarySoundEventAppCore):
    pass

app = Task2AppCore(
    name='DCASE 2017::Detection of rare sound events / Baseline System',
    params=params,
    system_desc=params.get('description'),
    system_parameter_set_id=params.get('active_set'),
    setup_label='Development setup',
    log_system_progress=params.get_path('general.log_system_progress'),
    show_progress_in_console=params.get_path('general.print_system_progress'),
    use_ascii_progress_bar=params.get_path('general.use_ascii_progress_bar')
)

# app.initialize()

[I] DCASE 2017::Detection of rare sound events / Baseline System
[I] 


In [5]:
#app.initialize()
app.feature_extraction()
app.feature_normalization()
app.system_training()

# app.system_testing()


[I] Feature extractor
[I] ==================================================
[I] 


[I]   DONE     [0:00:00.158139 ] [2987 items] 
[I] 
[I] Feature normalizer
[I] ==================================================
[I] 


           Feature method :   0%|          | 0/1 [00:00<?, ?it/s]
                                                                 
           Feature method :   0%|          | 0/1 [00:00<?, ?it/s]
                                                                 
           Feature method :   0%|          | 0/1 [00:00<?, ?it/s]
                                                                 

[I]   DONE     [0:00:00.007389 ] 
[I] 
[I] System training
[I] ==================================================
[I] 


                    Event :   0%|          | 0/3 [00:00<?, ?it/s]
                                                                 

[I]   DONE     [0:00:00.004338 ] 
[I] 


In [ ]:
# system_tesing を逐次実行で確認
from dcase_framework.metadata import MetaDataContainer, MetaDataItem
from dcase_framework.features import FeatureContainer, FeatureRepository, FeatureExtractor, FeatureNormalizer, \
    FeatureStacker, FeatureAggregator, FeatureMasker


from tqdm import tqdm
from six import iteritems

overwrite = True

fold_progress = tqdm(app._get_active_folds(),
                     desc="           {0: >15s}".format('Fold '),
                     file=sys.stdout,
                     leave=False,
                     miniters=1,
                     disable=app.disable_progress_bar,
                     ascii=app.use_ascii_progress_bar)

for fold in fold_progress:
    if app.log_system_progress:
        app.logger.info('  {title:<15s} [{fold:d}/{total:d}]'.format(title='Fold',
                                                                      fold=fold,
                                                                      total=len(fold_progress)))

    event_labels = app.dataset.event_labels
    print(event_labels)
    #if app.params.get_path('recognizer.active_events'):
    #    event_labels = list(set(event_labels).intersection(app.params.get_path('recognizer.active_events'))) -> None

    event_progress = tqdm(event_labels,
                          desc="           {0: >15s}".format('Event '),
                          file=sys.stdout,
                          leave=False,
                          miniters=1,
                          disable=app.disable_progress_bar,
                          ascii=app.use_ascii_progress_bar)
    
    single_file_per_fold = False
    if single_file_per_fold:
        current_result_file = app._get_result_filename(
            fold=fold,
            path=app.params.get_path('path.recognizer')
        )
        if not os.path.isfile(current_result_file) or overwrite:
            results = MetaDataContainer(filename=current_result_file)

    for event_label in event_progress:
        if not single_file_per_fold:
            current_result_file = app._get_result_filename(
                fold=fold,
                path=app.params.get_path('path.recognizer'),
                event_label=event_label
            )
            if not os.path.isfile(current_result_file) or overwrite:
                results = MetaDataContainer(filename=current_result_file)
                

        if not os.path.isfile(current_result_file) or overwrite:
            # Load class model container
#             model_filename = app._get_model_filename(
#                 fold=fold,
#                 path=app.params.get_path('path.learner'),
#                 event_label=event_label
#             )
            model_filename = '/store/projects/ml/mathG/DCASE2017/baseline/DCASE2017-baseline-system/applications/system/task2/learner/dataset_c17f65145b5c15a1a0afc148dc881423/feature_extractor_55ff217ecf31320b4d18e4e1d8f62a94/feature_stacker_d1b7b226d0bb278b6b3aab557e501d2c/feature_normalizer_86c6764b339414c628793552d084ff29/feature_aggregator_db31e2e54edd841b744da9dc385ab466/learner_d33fae90cf8ac2f03c749de5cd551af3/model_fold1_babycry.cpickle'

            
            if os.path.isfile(model_filename):
                model_container = app._get_learner(method=app.params.get_path('learner.method')).load(
                    filename=model_filename)
            else:
                message = '{name}: Model file not found [{file}]'.format(
                    name=app.__class__.__name__,
                    file=model_filename
                )

                app.logger.exception(message)
                raise IOError(message)


            item_progress = tqdm(app.dataset.test(fold, event_label=event_label),
                                 desc="           {0: >15s}".format('Testing '),
                                 file=sys.stdout,
                                 leave=False,
                                 disable=app.disable_progress_bar,
                                 ascii=app.use_ascii_progress_bar)

            # ================ debugger ======================
#             from IPython.core.debugger import Pdb; Pdb().set_trace()
            
            for item_id, item in enumerate(item_progress):
                if app.log_system_progress:
                    app.logger.info(
                        '  {title:<15s} [{item_id:3s}/{total:3s}] {item:<20s}'.format(
                            title='Testing',
                            item_id='{:d}'.format(item_id),
                            total='{:d}'.format(len(item_progress)),
                            item=os.path.split(item['file'])[-1])
                        )

                # Load features
                feature_filenames = app._get_feature_filename(
                    audio_file=item['file'],
                    path=app.params.get_path('path.feature_extractor')
                )

                # ================ debugger ======================
#                 from IPython.core.debugger import Pdb; Pdb().set_trace()

                feature_list = {}
                for method, feature_filename in iteritems(feature_filenames):
                    if os.path.isfile(feature_filename):
                        feature_list[method] = FeatureContainer().load(filename=feature_filename)
                    else:
                        message = '{name}: Features not found [{file}]'.format(
                            name=app.__class__.__name__,
                            file=item['file']
                        )

                        app.logger.exception(message)
                        raise IOError(message)


                        
                if hasattr(model_container, 'data_processor'):
                    # Leave feature and data processing to DataProcessor stored inside the model
                    feature_data = feature_list

                else:
                    # Backward compatibility mode
                    feature_data = model_container.feature_stacker.process(
                        feature_data=feature_list
                    )

                    # Normalize features
                    if model_container.feature_normalizer:
                        feature_data = model_container.feature_normalizer.normalize(feature_data)

                    # ================ debugger ======================
                    from IPython.core.debugger import Pdb; Pdb().set_trace()
                    # Aggregate features
                    if model_container.feature_aggregator:
                        feature_data = model_container.feature_aggregator.process(feature_data)

                # Frame probabilities
                frame_probabilities = model_container.predict(
                    feature_data=feature_data,
                )

                
                
                # Event recognizer
                current_result = app.EventRecognizer(
                    hop_length_seconds=model_container.params.get_path('hop_length_seconds'),
                    params=app.params.get_path('recognizer'),
                    class_labels=model_container.class_labels
                ).process(
                    frame_probabilities=frame_probabilities
                )

                if current_result:
                    for event in current_result:
                        event.file = app.dataset.absolute_to_relative(item['file'])
                        results.append(event)
                else:
                    results.append(MetaDataItem({'file': app.dataset.absolute_to_relative(item['file'])}))

            if not single_file_per_fold:
                # Save testing results
                results.save()


        
app.params.get_path('recognizer.event_handling') == 'event-dependent'
 

                     Fold :   0%|          | 0/1 [00:00<?, ?it/s]['babycry', 'glassbreak', 'gunshot']

                    Event :   0%|          | 0/3 [00:00<?, ?it/s]

                  Testing :   0%|          | 0/496 [00:00<?, ?it/s]> <ipython-input-6-028e652d3545>(143)<module>()
    141                     from IPython.core.debugger import Pdb; Pdb().set_trace()
    142                     # Aggregate features
--> 143                     if model_container.feature_aggregator:
    144                         feature_data = model_container.feature_aggregator.process(feature_data)
    145 



ipdb>  c


> /store/projects/ml/mathG/DCASE2017/baseline/DCASE2017-baseline-system/dcase_framework/features.py(1666)process()
-> start_frame = int(frame - numpy.floor(self.win_length_frames/2.0))


(Pdb)  c


> /store/projects/ml/mathG/DCASE2017/baseline/DCASE2017-baseline-system/dcase_framework/features.py(1681)process()
-> if 'mean' in self.recipe:


(Pdb)  current_frame.shpe


*** AttributeError: 'numpy.ndarray' object has no attribute 'shpe'


(Pdb)  current_frame.shape


(5, 40)


In [16]:
model_container.feature_aggregator.process(feature_data)['feat'][0].shape








(1501, 200)

In [13]:
feature_data['feat'][0].shape

(1501, 40)

In [40]:
feature_list['mel']['feat'][0].shape

(1501, 40)

In [41]:
feature_data = model_container.feature_stacker.process(
    feature_data=feature_list
)


In [42]:
fe

{'feat': [array([[ -4.9921308,  -6.8127418,  -8.27339  , ..., -11.935065 ,
          -12.281855 , -12.7022915],
         [ -5.31972  ,  -6.123804 ,  -7.168656 , ..., -11.131154 ,
          -11.4557295, -12.012918 ],
         [ -4.727575 ,  -6.4510207,  -7.4674263, ..., -11.562244 ,
          -11.813134 , -12.33866  ],
         ...,
         [ -5.136989 ,  -5.7218328,  -6.880217 , ..., -12.093683 ,
          -12.210219 , -12.4502325],
         [ -4.8826995,  -5.775437 ,  -7.178931 , ..., -11.99949  ,
          -12.115118 , -12.499706 ],
         [ -5.3415813,  -6.4713783,  -6.4889216, ..., -12.394382 ,
          -12.423141 , -12.574921 ]], dtype=float32)],
 'stat': None,
 'meta': {'parameters': {'fs': 44100,
   'win_length_seconds': 0.04,
   'win_length_samples': 1764,
   'hop_length_seconds': 0.02,
   'hop_length_samples': 882},
  'datetime': '2019-12-29 01:36:54',
  'audio_file': '/store/projects/ml/mathG/DCASE2017/baseline/DCASE2017-baseline-system/applications/data/TUT-rare-sound-ev

In [10]:
feature_data['feat'][0].shape

(1501, 40)

In [44]:
# Normalize features
if model_container.feature_normalizer:
    feature_data = model_container.feature_normalizer.normalize(feature_data)



In [11]:
data = feature_data['feat'][0] 

In [17]:
window = 10

current_frame = data[0: window]
current_frame.shape


(10, 40)

In [47]:
# Aggregate features
if model_container.feature_aggregator:
    feature_data = model_container.feature_aggregator.process(feature_data)

feature_data['feat'][0][0,:]

array([-0.36834243, -0.97110495, -1.75990413, -0.86744295, -0.61749806,
       -0.53168002, -0.06657085, -0.41225926, -0.5221846 , -0.65945249,
       -0.72783925, -0.91174275, -0.71849658, -0.13726441, -0.20630142,
       -0.00586321, -0.12863739,  0.08363921, -0.02463846, -0.01158372,
       -0.24910778, -0.33474139, -0.13847547, -0.40221948, -0.58989804,
       -0.34303652, -0.12239632, -0.16275208, -0.12377683, -0.18901634,
       -0.57300555, -0.95947006, -0.24280054,  0.62659029,  0.44254088,
       -0.26327022, -0.70951026, -0.66159259, -0.96034486, -1.09596488,
       -0.36834243, -0.97110495, -1.75990413, -0.86744295, -0.61749806,
       -0.53168002, -0.06657085, -0.41225926, -0.5221846 , -0.65945249,
       -0.72783925, -0.91174275, -0.71849658, -0.13726441, -0.20630142,
       -0.00586321, -0.12863739,  0.08363921, -0.02463846, -0.01158372,
       -0.24910778, -0.33474139, -0.13847547, -0.40221948, -0.58989804,
       -0.34303652, -0.12239632, -0.16275208, -0.12377683, -0.18

In [9]:
params.get_path('feature_aggregator')

{'enable': True,
 'aggregation_recipe': [{'method': 'flatten'}],
 'win_length_seconds': 0.1,
 'hop_length_seconds': 0.02,
 'win_length_frames': 5,
 'hop_length_frames': 1,
 '_hash': 'db31e2e54edd841b744da9dc385ab466'}